In [36]:
# All the packages and manual MERT implemented stuff
source("MERT_functions.R")

suppressMessages( library(arrow) )
suppressMessages( library(readxl) )

In [49]:
head(info_municipios,3)

cod_ibge,atividade_principal,pib_nominal,populacao,quantidade_total_vagas,vereadores,area,alunos_escola
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3500105,Demais serviços,1054541,35023,2002,9,411,2359
3500204,"Agricultura, inclusive apoio à agricultura e a pós colheita",99734,3571,458,9,211,478
3500303,Demais serviços,942708,35954,1471,13,474,3489


In [50]:
info_municipios <- read_excel("Data/info_municipios.xlsx")

info_municipios$cod_ibge <- factor(info_municipios$cod_ibge)
info_municipios$atividade_principal <- factor(info_municipios$atividade_principal)
info_municipios <- info_municipios %>% select(cod_ibge, atividade_principal, pib_nominal, populacao, quantidade_total_vagas, vereadores, area, alunos_escola)

df <- read_parquet("Data/data2010to2022_deflated.parquet")

df$cod_ibge <- factor(df$cod_ibge, levels = levels(info_municipios$cod_ibge))
df <- merge(df, info_municipios, on = "cod_ibge")

head(df, 3)

,cod_ibge,ds_municipio,ds_orgao,id_despesa,ano_exercicio,vl_despesa,atividade_principal,pib_nominal,populacao,quantidade_total_vagas,vereadores,area,alunos_escola
,<fct>,<chr>,<chr>,<chr>,<int>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3500105,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,0101,2010,395839.09,Demais serviços,1054541,35023,2002,9,411,2359
2,3500105,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,0301,2010,35329.42,Demais serviços,1054541,35023,2002,9,411,2359
3,3500105,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,1101,2010,18075650.35,Demais serviços,1054541,35023,2002,9,411,2359


In [34]:
df_grouped <- df %>% group_by(cod_ibge, ds_orgao, id_despesa) %>% summarise(
    ano_exercicio = list(ano_exercicio),
    vl_despesa = list(vl_despesa),
    .groups = "drop"
)

# Number of subjects
N <- nrow(df_grouped)
ids <- 1:N
df_grouped <- cbind(ids, df_grouped) %>% as.data.frame

head(df_grouped, 3)

,ids,cod_ibge,ds_orgao,id_despesa,ano_exercicio,vl_despesa
,<int>,<chr>,<chr>,<chr>,<list>,<list>
1,1,3500105,CENTRO UNIVERSITÁRIO DE ADAMANTINA,0101,"2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022","395839.1, 348831.7, 283607.7, 184368.7, 192036.1, 1580961.8, 1624274.9, 1683864.9, 1589171.1, 1602931.6, 1625335.3, 1346395.9, 1234774.1"
2,2,3500105,CENTRO UNIVERSITÁRIO DE ADAMANTINA,0301,"2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022","35329.42, 31010.99, 30304.01, 26272.16, 27109.21, 28489.60, 34558.81, 30639.90, 31634.26, 32609.79, 34321.61, 28667.90, 23820.15"
3,3,3500105,CENTRO UNIVERSITÁRIO DE ADAMANTINA,1101,"2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022","18075650, 15510074, 14545275, 13651045, 15308431, 16852914, 19364337, 21830750, 22110773, 24150379, 25402516, 21004560, 20206524"


In [ ]:
Ys <- df_grouped$vl_despesa

# Recover all the matrices Xi from every subject (intercept in this case)
Xis <- apply(df_grouped, 1, function(row){
    dfi <- as.data.frame(row) %>% unnest(cols = c("vl_despesa", "ano_exercicio"))
    Xi <- model.matrix(Y ~ t, data = dfi)
    list(Xi)
}) %>% unlist(recursive = FALSE)

# Recover all the matrices Zi from every subject according to the desired model (random intercept in this case)
Zis <- apply(df_grouped, 1, function(row){
    dfi <- as.data.frame(row) %>% unnest(cols = c("Y", "t"))
    Zi <- model.matrix(Y ~ 1, data = dfi)
    list(Zi)
}) %>% unlist(recursive = FALSE)